In [3]:
import jax
jdev = jax.devices()[0]
print(jdev)
x=jax.numpy.arange(1,2,0.3)

import flax
import optax
import jax.numpy as jnp
import flax.linen as nn

import pickle
with open('pickled_data.pickle','rb') as f:
    pickled_data=pickle.load(f)
print(pickled_data.keys())

x_train=pickled_data['x_train']
y_train_one_hot=pickled_data['y_train_one_hot']
x_test=pickled_data['x_test']
y_test_one_hot=pickled_data['y_test_one_hot']

x_train = jax.device_put(x_train,device=jdev)
y_train_one_hot = jax.device_put(y_train_one_hot,device=jdev)

x_test = jax.device_put(x_test,device=jdev)
y_test_one_hot = jax.device_put(y_test_one_hot,device=jdev)

class CNN(nn.Module):
    dense_features: int

    @nn.compact
    def __call__(self, x):
        x = nn.Conv(features=32, kernel_size=(3, 3))(x)
        x = nn.leaky_relu(x)
        x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
        x = nn.Conv(features=48, kernel_size=(3, 3))(x)
        x = nn.leaky_relu(x)
        x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
        x = x.reshape((x.shape[0], -1))  # flatten
        x = nn.Dense(features=self.dense_features)(x)
        x = nn.leaky_relu(x)
        x = nn.Dense(features=10)(x)
        x = nn.log_softmax(x)
        return x




No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


TFRT_CPU_0
dict_keys(['x_train', 'y_train_one_hot', 'x_test', 'y_test_one_hot'])


In [4]:
dense_features=64
# instantiate the model and it's parameters
cnn = CNN(dense_features=dense_features,name='src')
parameters = cnn.init(jax.random.PRNGKey(0), x_train)

In [5]:
print(nn.tabulate(CNN(dense_features=dense_features,name='src'),rngs=jax.random.PRNGKey(0))(x_train))


                                  CNN Summary                                   
┏━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ path    ┃ module ┃ inputs            ┃ outputs           ┃ params            ┃
┡━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│         │ CNN    │ uint8[60000,28,2… │ float32[60000,10] │                   │
├─────────┼────────┼───────────────────┼───────────────────┼───────────────────┤
│ Conv_0  │ Conv   │ uint8[60000,28,2… │ float32[60000,28… │ bias: float32[32] │
│         │        │                   │                   │ kernel:           │
│         │        │                   │                   │ float32[3,3,1,32] │
│         │        │                   │                   │                   │
│         │        │                   │                   │ 320 (1.3 KB)      │
├─────────┼────────┼───────────────────┼───────────────────┼───────────────────┤
│ Conv_1  │ Conv   │ float3

In [9]:
# prepare loss function
@jax.jit
def cnn_train_loss_fn(params):
    train_logits = cnn.apply(params, x_train)
    train_loss = jnp.mean(optax.softmax_cross_entropy(logits=train_logits, labels=y_train_one_hot))
    return train_loss, train_logits

def cnn_test_accuracy_fn(params):
    test_logits = cnn.apply(params, x_test)
    test_output = jnp.argmax(test_logits, -1)
    misclassified_item_count = jnp.sum( test_output != y_test_one_hot)
    test_accuracy = jnp.mean( test_output == x_test_one_hot )
    return test_accuracy, misclassified_item_count

# get a gradient function of the loss function
cnn_train_loss_grad_fn = jax.jit(jax.value_and_grad(cnn_train_loss_fn, has_aux=True))
# example evaluation of the loss gradient function
(loss, logits), grads = cnn_train_loss_grad_fn(parameters)



Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/cpu. Invalid point: 2023-01-07T15:36:28.401Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/memory. Invalid point: 2023-01-07T15:36:28.401Z


In [7]:
import optax
tx = optax.adam(learning_rate=3e-3)
tx_state = tx.init(parameters)

In [8]:
import neptune.new as neptune
meta_name = f"dense=[{dense_features}]"
run = neptune.init_run(
    project="gre/refresh-cnn",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJkZjQyNGQ0Ni1hODgwLTQzMDEtOGQ1NS1lNzczNWFlMTE4OWQifQ==",
    custom_run_id=meta_name,
    tags=meta_name
    )  # your credentials

metadata = {"optimizer": "Adam"}
run["metadata"] = metadata
# epoch = 0
# with open('snapshot','rb') as f:
#     snapshot = f.read()
# parameters = flax.serialization.from_bytes(target = parameters, encoded_bytes=snapshot)
run["meta/tag"].log(meta_name)

https://app.neptune.ai/gre/refresh-cnn/e/REF-9


Info (NVML): NVML Shared Library Not Found. GPU usage metrics may not be reported. For more information, see https://docs.neptune.ai/help/nvml_error/


Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [10]:
epoch=0
for cycle in range(150):
    epoch +=1
    (loss, logits), grads = cnn_train_loss_grad_fn(parameters)
    accuracy, misses =cnn_test_accuracy_fn(parameters)
    try:
        run["train/epoch"].log(epoch)
        run["train/train-loss"].log(loss)
        run["train/test-accuracy"].log(accuracy)
        run["train/test-misses"].log(misses)
    except:
        pass
    neglog_accuracy = - jnp.log10(1.00-accuracy)
    run["train/test-neglog-accuracy"].log(neglog_accuracy)
    print(f'epoch {epoch} : loss={loss: 0.3f}, misses={misses}, accuracy={100*accuracy: 0.3f}% -> neglog={neglog_accuracy:0.3f}')
    updates, tx_state = tx.update(grads, tx_state)
    parameters = optax.apply_updates(parameters, updates)
    snapshot = flax.serialization.to_bytes(parameters)
    with open('snapshot','wb') as f:
        f.write(snapshot)
run["eval/neglog"] = neglog_accuracy
run.stop()


Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/cpu. Invalid point: 2023-01-07T15:37:28.448Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/memory. Invalid point: 2023-01-07T15:37:28.448Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/memory. Invalid point: 2023-01-07T15:37:38.466Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/cpu. Invalid point: 2023-01-07T15:37:38.466Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/memory. Invalid point: 2023-01-07T15:37:48.477Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/cpu. Invalid point: 2023-01-

ValueError: Incompatible shapes for broadcasting: shapes=[(10000,), (10000, 10)]